### 머신러닝 첫번째 - 데이터 정규화

오라클 접속

In [2]:
import oracledb

# Oracle Instant Client 라이브러리 경로
client_dir = r"C:\\Users\\user\\Desktop\\KSM\\Tools\\instantclient-basic-windows.x64-19.25.0.0.0dbru\\instantclient_19_25"
oracledb.init_oracle_client(lib_dir=client_dir)

# 접속 정보 입력
username = "libra"
password = "ksm0923"
dsn = "localhost:1521/XE"

# 접속 시도
try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = conn.cursor()
    print("🔗 Oracle DB 접속 성공")
except Exception as e:
    print("❌ Oracle DB 연결 실패:", e)

🔗 Oracle DB 접속 성공


연도별 NUM01~05 데이터를 하나의 테이블로 합치기 -> TD_"연도" 태이블 생성 

In [9]:
import pandas as pd

# 🔁 연도 및 공통 컬럼 설정
years = range(2014, 2025)
common_cols = ['YR', 'ID', 'SNM', 'STYP', 'FND', 'RGN', 'USC']

for year in years:
    dfs = []
    for i in range(1, 6):  # NUM01 ~ NUM05
        table = f"NUM0{i}_{year}"

        try:
            df = pd.read_sql(f"SELECT * FROM {table}", conn)
            df.columns = [col.upper() for col in df.columns]  # 대소문자 통일
            df['YR'] = year  # 혹시 빠져 있으면 삽입

            if i > 1:
                df = df.drop(columns=[col for col in common_cols if col in df.columns])

            dfs.append(df)

        except Exception as e:
            print(f"❌ {table} 불러오기 실패: {e}")

    # 🧱 테이블 병합 및 저장
    if dfs:
        df_td = pd.concat(dfs, axis=1)
        td_table_name = f"TD_{year}"

        try:
            # 오라클 테이블 생성 (SQLAlchemy 없이 oracledb만으로 직접 실행)
            columns = ', '.join([f'"{col}" CLOB' if df_td[col].dtype == 'object' else f'"{col}" FLOAT' for col in df_td.columns])
            create_sql = f'CREATE TABLE "{td_table_name}" ({columns})'

            try:
                cursor.execute(f'DROP TABLE "{td_table_name}"')
            except:
                pass  # 존재 안 할 경우 무시

            cursor.execute(create_sql)

            # 레코드 입력
            rows = [tuple(row) for row in df_td.itertuples(index=False, name=None)]
            placeholders = ', '.join([':' + str(i+1) for i in range(len(df_td.columns))])
            insert_sql = f'INSERT INTO "{td_table_name}" VALUES ({placeholders})'
            cursor.executemany(insert_sql, rows)
            conn.commit()

            print(f"📦 Oracle 저장 완료: {td_table_name} ({df_td.shape})")

        except Exception as e:
            print(f"❌ Oracle 저장 실패: {td_table_name} → {e}")

C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_285

📦 Oracle 저장 완료: TD_2014 ((461, 139))


C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_285

📦 Oracle 저장 완료: TD_2015 ((461, 139))


C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_285

...

📦 Oracle 저장 완료: TD_2022 ((461, 169))


C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_285

📦 Oracle 저장 완료: TD_2023 ((461, 169))


C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\414588176.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_285

📦 Oracle 저장 완료: TD_2024 ((461, 168))


NUM00_연도 테이블을 기준으로 평가점수가 존재하는 데이터만 필터링하여 SCORE_ALL_"연도" 테이블 생성

In [ ]:
# 오류코드

import pandas as pd

for year in range(2014, 2025):
    try:
        # 📥 NUM00 테이블 불러오기
        df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
        df_num.columns = [col.upper() for col in df_num.columns]
        df_num["YR"] = year

        # 📥 TD 테이블 불러오기
        df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)
        df_td.columns = [col.upper() for col in df_td.columns]

        # 🔗 병합: NUM00 기준 LEFT JOIN
        df_merge = pd.merge(df_num, df_td, on=["SNM", "YR"], how="left")

        # 🧹 중복 컬럼 제거: *_X, *_Y → 우측 TD 컬럼 제거
        df_merge = df_merge.loc[:, ~df_merge.columns.duplicated()]
        df_merge.columns = [col.replace('_X', '').replace('_Y', '') for col in df_merge.columns]

        # 🔧 데이터 타입 정리: 숫자형 → float 변환 / 문자형 → str
        for col in df_merge.select_dtypes(include=['float', 'int']).columns:
            df_merge[col] = pd.to_numeric(df_merge[col], errors='coerce')

        for col in df_merge.select_dtypes(include='object').columns:
            df_merge[col] = df_merge[col].astype(str)

        # 🏛️ Oracle 테이블 생성
        table_name = f'SCORE_ALL_{year}'

        try:
            cursor.execute(f'DROP TABLE "{table_name}"')
        except:
            pass  # 없으면 무시

        col_defs = ', '.join([
            f'"{col}" CLOB' if df_merge[col].dtype == object else f'"{col}" FLOAT'
            for col in df_merge.columns
        ])
        cursor.execute(f'CREATE TABLE "{table_name}" ({col_defs})')

        # 🔽 INSERT
        rows = [tuple(row) for row in df_merge.itertuples(index=False, name=None)]
        placeholders = ', '.join([f':{i+1}' for i in range(len(df_merge.columns))])
        cursor.executemany(f'INSERT INTO "{table_name}" VALUES ({placeholders})', rows)
        conn.commit()

        print(f"✅ 저장 완료: {table_name} → {df_merge.shape[0]}행 × {df_merge.shape[1]}컬럼")

    except Exception as e:
        print(f"❌ 저장 실패: SCORE_ALL_{year} → {e}")

C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 저장 완료: SCORE_ALL_2014 → 34행 × 141컬럼


C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 저장 완료: SCORE_ALL_2015 → 36행 × 141컬럼


C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


...

✅ 저장 완료: SCORE_ALL_2022 → 57행 × 171컬럼


C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 저장 완료: SCORE_ALL_2023 → 57행 × 171컬럼


C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\3214456646.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 저장 완료: SCORE_ALL_2024 → 57행 × 170컬럼


인풋 후보군 컬럼만 남겨두고 전체 삭제, 모든연도를 합친 최종 테이블 생성

In [19]:
import pandas as pd

# 🎯 입력 변수 약문명 (21개)
input_abbr = [
    'BCNT_SUM', 'BPS', 'ABI', 'LAS', 'TRS', 'TPC', 'APS',
    'UBGT', 'USTL', 'MCT', 'CPS',
    'LUC', 'LU_SUM', 'LB_SUM', 'EDC', 'EP', 'LPS', 'VPS',
    'SPK', 'LPK', 'EHS'
]

# 🎯 메타/타깃 변수 (9개)
meta_cols = ['YR', 'RK', 'SNM', 'SCR', 'ID', 'STYP', 'FND', 'RGN', 'USC']

# 🙌 컬럼 유지 목록
keep_set = set(input_abbr + meta_cols)

filtered_dfs = []

for year in range(2014, 2025):
    try:
        df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
        df.columns = [col.upper() for col in df.columns]

        # ✅ 접미어 기준 필터링
        selected_cols = [col for col in df.columns if any(col.endswith(k) or col == k for k in keep_set)]
        df = df[selected_cols]

        # ✅ 누락 컬럼 보완
        for col in keep_set:
            if col not in df.columns:
                df[col] = None

        # ✅ 컬럼 순서 정렬
        df = df[[col for col in meta_cols + input_abbr]]
        filtered_dfs.append(df)

        print(f"✅ 필터링 완료: SCORE_ALL_{year} → {df.shape}")

    except Exception as e:
        print(f"❌ 실패: {year} → {e}")

# 📦 전체 병합
df_filtered = pd.concat(filtered_dfs, ignore_index=True)

# 🔧 데이터 타입 정제
for col in df_filtered.select_dtypes(include=['float', 'int']).columns:
    df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')
for col in df_filtered.select_dtypes(include='object').columns:
    df_filtered[col] = df_filtered[col].astype(str)

# 🏛️ Oracle 저장: SCORE_FILTERED
try:
    cursor.execute('DROP TABLE "SCORE_FILTERED"')
except:
    pass

col_defs = ', '.join([
    f'"{col}" CLOB' if df_filtered[col].dtype == object else f'"{col}" FLOAT'
    for col in df_filtered.columns
])
cursor.execute(f'CREATE TABLE "SCORE_FILTERED" ({col_defs})')

rows = [tuple(row) for row in df_filtered.itertuples(index=False, name=None)]
placeholders = ', '.join([f':{i+1}' for i in range(len(df_filtered.columns))])
cursor.executemany(f'INSERT INTO "SCORE_FILTERED" VALUES ({placeholders})', rows)
conn.commit()

print(f"🎯 SCORE_FILTERED 저장 완료 → {df_filtered.shape[0]}행 × {df_filtered.shape[1]}컬럼")

C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\Ap

✅ 필터링 완료: SCORE_ALL_2014 → (34, 30)
✅ 필터링 완료: SCORE_ALL_2015 → (36, 30)
✅ 필터링 완료: SCORE_ALL_2016 → (36, 30)
✅ 필터링 완료: SCORE_ALL_2017 → (36, 30)
✅ 필터링 완료: SCORE_ALL_2018 → (35, 30)
✅ 필터링 완료: SCORE_ALL_2019 → (64, 30)
✅ 필터링 완료: SCORE_ALL_2020 → (61, 30)
✅ 필터링 완료: SCORE_ALL_2021 → (60, 30)


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\Ap

✅ 필터링 완료: SCORE_ALL_2022 → (57, 30)
✅ 필터링 완료: SCORE_ALL_2023 → (57, 30)
✅ 필터링 완료: SCORE_ALL_2024 → (57, 30)


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2024841746.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM SCORE_ALL_{year}', conn)


🎯 SCORE_FILTERED 저장 완료 → 533행 × 30컬럼


In [20]:
import pandas as pd

# 🔁 Oracle에서 SCORE_FILTERED 테이블 불러오기
df_filtered = pd.read_sql('SELECT * FROM SCORE_FILTERED', conn)

# 💾 CSV로 저장
csv_path = r"C:\Users\user\Desktop\SCORE_FILTERED.csv"
df_filtered.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"✅ CSV 저장 완료 → {csv_path}")

C:\Users\user\AppData\Local\Temp\ipykernel_28588\2325416135.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_filtered = pd.read_sql('SELECT * FROM SCORE_FILTERED', conn)


✅ CSV 저장 완료 → C:\Users\user\Desktop\SCORE_FILTERED.csv


In [24]:
for year in range(2014, 2025):
    try:
        table_name = f'SCORE_ALL_{year}'
        df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
        df.columns = [col.upper() for col in df.columns]

        print(f"\n🗓️ {table_name} 상위 5행:")
        display(df.head())

    except Exception as e:
        print(f"❌ {table_name} 불러오기 실패: {e}")



🗓️ SCORE_ALL_2014 상위 5행:


C:\Users\user\AppData\Local\Temp\ipykernel_28588\1955630010.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)


,YR,RK,SNM,SCR,ID,STYP,FND,RGN,USC,BR_TCNT_KB,...,SPK_SCC,SPK_SPK,LPK_LFT2,LPK_LPT2,LPK_SCP,LPK_LPK,EHS_EH,EHS_LFT,EHS_LPT,EHS_EHS
0,2014.0,3,한국과학기술원,49.88,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2014.0,4,고려대학교,49.84,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2014.0,5,한양대학교,48.63,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2014.0,1,서울대학교,66.06,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2014.0,2,연세대학교,50.95,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan



🗓️ SCORE_ALL_2015 상위 5행:


C:\Users\user\AppData\Local\Temp\ipykernel_28588\1955630010.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)


...

C:\Users\user\AppData\Local\Temp\ipykernel_28588\1955630010.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)


,YR,RK,SNM,SCR,ID,STYP,FND,RGN,USC,BR_TCNT_KB,...,SPK_SCC,SPK_SPK,LPK_LFT2,LPK_LPT2,LPK_SCC,LPK_LPK,EHS_EH,EHS_LFT,EHS_LPT,EHS_EHS
0,2023.0,2,고려대학교,79.8,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2023.0,1,서울대학교,86.6,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2023.0,5,한국과학기술원,78.2,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2023.0,6,한양대학교,76.7,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2023.0,3,연세대학교,79.8,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


C:\Users\user\AppData\Local\Temp\ipykernel_28588\1955630010.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)



🗓️ SCORE_ALL_2024 상위 5행:


,YR,RK,SNM,SCR,ID,STYP,FND,RGN,USC,BR_TCNT_KB,...,SPK_SCC,SPK_SPK,LPK_LFT2,LPK_LPT2,LPK_SCC,LPK_LPK,EHS_EH,EHS_LFT,EHS_LPT,EHS_EHS
0,2024.0,2,연세대학교,80.2,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2024.0,1,서울대학교,86.7,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2024.0,5,한국과학기술원,78,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2024.0,6,한양대학교,76.7,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2024.0,3,고려대학교,80,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [31]:
import pandas as pd

for year in range(2014, 2025):
    print(f"\n🔄 {year}년 처리 중...")

    # 📥 NUM00 불러오기
    df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
    df_num.columns = [col.upper() for col in df_num.columns]

    # 📥 TD 불러오기
    df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)
    df_td.columns = [col.upper() for col in df_td.columns]

    # 🔧 SNM 정제
    df_num["SNM"] = df_num["SNM"].astype(str).str.strip().str.upper()
    df_td["SNM"] = df_td["SNM"].astype(str).str.strip().str.upper()

    # 🔍 TD 필터링
    td_filtered = df_td[df_td["SNM"].isin(df_num["SNM"])].copy()

    # ⚠️ 누락된 SNM 확인
    unmatched_snms = df_num[~df_num["SNM"].isin(df_td["SNM"])]["SNM"].tolist()
    if unmatched_snms:
        print(f"⚠️ [경고] {year}년: NUM00에 있는 SNM 중 {len(unmatched_snms)}개가 TD에 없음!")
        print(f"   누락된 SNM 목록 ➜ {unmatched_snms}")

    # 🧹 순서 정렬
    td_filtered = td_filtered.set_index("SNM").reindex(df_num["SNM"].values).reset_index()

    # ✅ 중복 컬럼 제거
    overlap_cols = [col for col in td_filtered.columns if col in df_num.columns and col != "SNM"]
    td_filtered = td_filtered.drop(columns=overlap_cols)

    # 🔗 병합
    df_merge = pd.concat([df_num.reset_index(drop=True), td_filtered.drop(columns="SNM")], axis=1)

    # 💾 저장
    csv_path = f"SCORE_ALL_{year}_MERGED.csv"
    df_merge.to_csv(csv_path, index=False, encoding='utf-8-sig')

    print(f"✅ {year}년 병합 완료 → {csv_path}, {df_merge.shape[0]}행 × {df_merge.shape[1]}컬럼")


🔄 2014년 처리 중...


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 2014년 병합 완료 → SCORE_ALL_2014_MERGED.csv, 34행 × 141컬럼

🔄 2015년 처리 중...


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 2015년 병합 완료 → SCORE_ALL_2015_MERGED.csv, 36행 × 141컬럼

🔄 2016년 처리 중...


...

✅ 2022년 병합 완료 → SCORE_ALL_2022_MERGED.csv, 57행 × 171컬럼

🔄 2023년 처리 중...


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 2023년 병합 완료 → SCORE_ALL_2023_MERGED.csv, 57행 × 171컬럼

🔄 2024년 처리 중...


C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num = pd.read_sql(f"SELECT * FROM NUM00_{year}", conn)
C:\Users\user\AppData\Local\Temp\ipykernel_28588\2069373574.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_td = pd.read_sql(f"SELECT * FROM TD_{year}", conn)


✅ 2024년 병합 완료 → SCORE_ALL_2024_MERGED.csv, 57행 × 170컬럼
